<a href="https://colab.research.google.com/github/Malanidhruv/Python-projects/blob/main/Technical%20strategy/Final_strategy_for_NIFTY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pya3
%pip install nsetools

## **NIFTY LOW VOLATILITY**

In [ ]:
from pya3 import *
from datetime import datetime, timedelta
from nsetools import Nse
import holidays
from google.colab import drive

def get_user_credentials():
    """Prompt the user for their credentials and return them."""
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice(user_id, api_key):
    """Initialize the AliceBlue session."""
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()  # Log in to AliceBlue
    return alice

def get_market_holidays(year):
    """Return a list of market holidays for the given year."""
    market_holidays = holidays.India(years=year)
    # Add Mahashivratri as a market holiday on 08-03-2024
    market_holidays.append(datetime(2024, 11, 20).date())
    return market_holidays

def find_previous_trading_day(current_date, market_holidays):
    """Find the previous trading day, excluding weekends and holidays."""
    previous_trading_day = current_date - timedelta(days=1)
    while previous_trading_day.weekday() >= 5 or previous_trading_day in market_holidays:
        previous_trading_day -= timedelta(days=1)
    return previous_trading_day

def fetch_historical_data(alice, instrument, date):
    """Fetch historical data for the given instrument on the specified date."""
    from_datetime = datetime.combine(date, datetime.min.time())
    to_datetime = datetime.combine(date, datetime.max.time())
    interval = "D"
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    return historical_data

def next_thursday():
    """Find the next Thursday from the current date."""
    today = datetime.today().date()
    days_ahead = 3 - today.weekday()
    if days_ahead <= 0:  # Target day has already happened this week
        days_ahead += 7
    return today + timedelta(days=days_ahead)

def calculate_strike_prices(closing_price):
    """Calculate call and put option strike prices based on the closing price."""
    ce_strike_price = int(closing_price / 50) * 50
    pe_strike_price = ce_strike_price + 50
    return ce_strike_price, pe_strike_price

def construct_option_symbols(expiry_date, ce_strike_price, pe_strike_price):
    """Construct option symbols for call and put options."""
    symbol_ce = f'NIFTY{expiry_date.strftime("%d%b%y")}C{ce_strike_price}'
    symbol_pe = f'NIFTY{expiry_date.strftime("%d%b%y")}P{pe_strike_price}'
    return symbol_ce, symbol_pe

def fetch_highest_high(alice, symbol, from_datetime, to_datetime, interval):
    """Fetch the highest high for the given symbol."""
    instrument = alice.get_instrument_by_symbol('NFO', symbol)
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    highest_high = historical_data['high'].max()
    return round(float(highest_high))

def fetch_lowest_low(alice, symbol, from_datetime, to_datetime, interval):
    """Fetch the lowest low for the given symbol."""
    instrument = alice.get_instrument_by_symbol('NFO', symbol)
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    lowest_low = historical_data['low'].min()  # Use min() to get the lowest low
    return round(float(lowest_low))

def place_order(alice, transaction_type, symbol, price_value, quantity=25):
    """Place an order for the given symbol."""
    if not isinstance(price_value, (float, int)):
        raise ValueError(f"Price must be a float or int, got {type(price_value).__name__}")

    price_value = float(price_value)  # Ensure price_value is a float

    return alice.place_order(
        transaction_type=transaction_type,
        instrument=alice.get_instrument_by_symbol('NFO', symbol),
        quantity=quantity,
        order_type=OrderType.StopLossLimit,
        product_type=ProductType.BracketOrder,
        price= price_value,
        trigger_price=price_value,
        stop_loss=15.0,
        square_off=50.0,
        trailing_sl=0.0,
        is_amo=True,
        order_tag='order1'
    )

def main():

    # Step 1: Mount Google Drive
    drive.mount('/content/drive')

    # Step 2: Load the CSV file
    nfo_file_path = '/content/drive/My Drive/NFO.csv'  # Change this path accordingly
    nfo_data = pd.read_csv(nfo_file_path)

    # Get user credentials and initialize AliceBlue
    user_id, api_key = get_user_credentials()
    alice = initialize_alice(user_id, api_key)

    # Create an instance of Nse
    nse = Nse()

    # Get market holidays for the current year
    current_date = datetime.now()
    market_holidays = get_market_holidays(current_date.year)

    # Find the previous trading day
    previous_trading_day = find_previous_trading_day(current_date, market_holidays)
    print(f"Previous Trading Day: {previous_trading_day}")

    # Fetch historical data
    instrument = alice.get_instrument_by_symbol('NFO', 'NIFTY26DEC24F')
    historical_data = fetch_historical_data(alice, instrument, previous_trading_day)

    last_close_price = historical_data['close'].iloc[-1]
    print("Closing Price of the Last Minute:", last_close_price)

    # Calculate strike prices
    ce_strike_price, pe_strike_price = calculate_strike_prices(last_close_price)

    # Get the next Thursday for expiry date
    expiry_date = next_thursday()

    # Construct option symbols
    symbol_ce, symbol_pe = construct_option_symbols(expiry_date, ce_strike_price, pe_strike_price)
    print("CE Option Symbol:", symbol_ce)
    print("PE Option Symbol:", symbol_pe)

    # Define from_datetime, to_datetime, and interval for fetching highs and lows
    from_datetime = datetime.combine(previous_trading_day, datetime.min.time())
    to_datetime = datetime.combine(previous_trading_day, datetime.max.time())
    interval = "D"

    # Get the highest high and place orders for CE
    highest_high_ce = fetch_highest_high(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Highest High CE Value:", highest_high_ce)
    print(place_order(alice, TransactionType.Sell, symbol_ce, highest_high_ce))

    # Get the highest high and place orders for PE
    highest_high_pe = fetch_highest_high(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Highest High PE Value:", highest_high_pe)
    print(place_order(alice, TransactionType.Sell, symbol_pe, highest_high_pe))

    # Get the lowest low and place orders for CE
    lowest_low_ce = fetch_lowest_low(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low CE Value:", lowest_low_ce)
    print(place_order(alice, TransactionType.Buy, symbol_ce, lowest_low_ce))

    # Get the lowest low and place orders for PE
    lowest_low_pe = fetch_lowest_low(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low PE Value:", lowest_low_pe)
    print(place_order(alice, TransactionType.Buy, symbol_pe, lowest_low_pe))

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter your user ID: 1141826
Enter your API key: nGhoLNREiDSHvBgjyGGQhltRljzoQX9XqvbLoAXoP6brZivMUymtMumvpK5bvqlkXyGcqVXbtmfanyXqixyRSo3DVSLdusxtTfYZIi9n0afG4ixXUOg0jS2uVOKm6kWY
Previous Trading Day: 2024-11-28 06:43:59.876799
Closing Price of the Last Minute: 24115.25
CE Option Symbol: NIFTY05Dec24C24100
PE Option Symbol: NIFTY05Dec24P24150
Rounded Highest High CE Value: 375
{'stat': 'Ok', 'NOrdNo': '24112900138902'}
Rounded Highest High PE Value: 311
{'stat': 'Ok', 'NOrdNo': '24112900138912'}
Rounded Lowest Low CE Value: 126
{'stat': 'Ok', 'NOrdNo': '24112900138922'}
Rounded Lowest Low PE Value: 77
{'stat': 'Ok', 'NOrdNo': '24112900138930'}


# **NIFTY HIGH VOLATILITY**

In [ ]:
from pya3 import *
from datetime import datetime, timedelta
from nsetools import Nse
import holidays
from google.colab import drive

def get_user_credentials():
    """Prompt the user for their credentials and return them."""
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice(user_id, api_key):
    """Initialize the AliceBlue session."""
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()  # Log in to AliceBlue
    return alice

def get_market_holidays(year):
    """Return a list of market holidays for the given year."""
    market_holidays = holidays.India(years=year)
    # Add Mahashivratri as a market holiday on 08-03-2024
    market_holidays.append(datetime(2024, 11, 20).date())
    return market_holidays

def find_previous_trading_day(current_date, market_holidays):
    """Find the previous trading day, excluding weekends and holidays."""
    previous_trading_day = current_date - timedelta(days=1)
    while previous_trading_day.weekday() >= 5 or previous_trading_day in market_holidays:
        previous_trading_day -= timedelta(days=1)
    return previous_trading_day

def fetch_historical_data(alice, instrument, date):
    """Fetch historical data for the given instrument on the specified date."""
    from_datetime = datetime.combine(date, datetime.min.time())
    to_datetime = datetime.combine(date, datetime.max.time())
    interval = "D"
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    return historical_data

def next_thursday():
    """Find the next Thursday from the current date."""
    today = datetime.today().date()
    days_ahead = 3 - today.weekday()
    if days_ahead <= 0:  # Target day has already happened this week
        days_ahead += 7
    return today + timedelta(days=days_ahead)

def calculate_strike_prices(closing_price):
    """Calculate call and put option strike prices based on the closing price."""
    ce_strike_price = int(closing_price / 50) * 50
    pe_strike_price = ce_strike_price + 50
    return ce_strike_price, pe_strike_price

def construct_option_symbols(expiry_date, ce_strike_price, pe_strike_price):
    """Construct option symbols for call and put options."""
    symbol_ce = f'NIFTY{expiry_date.strftime("%d%b%y")}C{ce_strike_price}'
    symbol_pe = f'NIFTY{expiry_date.strftime("%d%b%y")}P{pe_strike_price}'
    return symbol_ce, symbol_pe

def fetch_highest_high(alice, symbol, from_datetime, to_datetime, interval):
    """Fetch the highest high for the given symbol."""
    instrument = alice.get_instrument_by_symbol('NFO', symbol)
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    highest_high = historical_data['high'].max()
    return round(float(highest_high))

def fetch_lowest_low(alice, symbol, from_datetime, to_datetime, interval):
    """Fetch the lowest low for the given symbol."""
    instrument = alice.get_instrument_by_symbol('NFO', symbol)
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    lowest_low = historical_data['low'].min()  # Use min() to get the lowest low
    return round(float(lowest_low))

def place_order(alice, transaction_type, symbol, price_value, quantity=25):
    """Place an order for the given symbol."""
    if not isinstance(price_value, (float, int)):
        raise ValueError(f"Price must be a float or int, got {type(price_value).__name__}")

    price_value = float(price_value)  # Ensure price_value is a float

    return alice.place_order(
        transaction_type=transaction_type,
        instrument=alice.get_instrument_by_symbol('NFO', symbol),
        quantity=quantity,
        order_type=OrderType.StopLossLimit,
        product_type=ProductType.BracketOrder,
        price= price_value,
        trigger_price=price_value,
        stop_loss=15.0,
        square_off=50.0,
        trailing_sl=0.0,
        is_amo=True,
        order_tag='order1'
    )

def main():
    # Step 1: Mount Google Drive
    drive.mount('/content/drive')

    # Step 2: Load the CSV file
    nfo_file_path = '/content/drive/My Drive/NFO.csv'  # Change this path accordingly
    nfo_data = pd.read_csv(nfo_file_path)

    # Get user credentials and initialize AliceBlue
    user_id, api_key = get_user_credentials()
    alice = initialize_alice(user_id, api_key)

    # Create an instance of Nse
    nse = Nse()

    # Get market holidays for the current year
    current_date = datetime.now()
    market_holidays = get_market_holidays(current_date.year)

    # Find the previous trading day
    previous_trading_day = find_previous_trading_day(current_date, market_holidays)
    print(f"Previous Trading Day: {previous_trading_day}")

    # Fetch historical data
    instrument = alice.get_instrument_by_symbol('NFO', 'NIFTY26DEC24F')
    historical_data = fetch_historical_data(alice, instrument, previous_trading_day)

    last_close_price = historical_data['close'].iloc[-1]
    print("Closing Price of the Last Minute:", last_close_price)

    # Calculate strike prices
    ce_strike_price, pe_strike_price = calculate_strike_prices(last_close_price)

    # Get the next Thursday for expiry date
    expiry_date = next_thursday()

    # Construct option symbols
    symbol_ce, symbol_pe = construct_option_symbols(expiry_date, ce_strike_price, pe_strike_price)
    print("CE Option Symbol:", symbol_ce)
    print("PE Option Symbol:", symbol_pe)

    # Define from_datetime, to_datetime, and interval for fetching highs and lows
    from_datetime = datetime.combine(previous_trading_day, datetime.min.time())
    to_datetime = datetime.combine(previous_trading_day, datetime.max.time())
    interval = "D"

    # Get the highest high and place orders for CE
    highest_high_ce = fetch_highest_high(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Highest High CE Value:", highest_high_ce)
    print(place_order(alice, TransactionType.Buy, symbol_ce, highest_high_ce))

    # Get the highest high and place orders for PE
    highest_high_pe = fetch_highest_high(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Highest High PE Value:", highest_high_pe)
    print(place_order(alice, TransactionType.Buy, symbol_pe, highest_high_pe))

    # Get the lowest low and place orders for CE
    lowest_low_ce = fetch_lowest_low(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low CE Value:", lowest_low_ce)
    print(place_order(alice, TransactionType.Sell, symbol_ce, lowest_low_ce))

    # Get the lowest low and place orders for PE
    lowest_low_pe = fetch_lowest_low(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low PE Value:", lowest_low_pe)
    print(place_order(alice, TransactionType.Sell, symbol_pe, lowest_low_pe))

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter your user ID: 1141826
Enter your API key: nGhoLNREiDSHvBgjyGGQhltRljzoQX9XqvbLoAXoP6brZivMUymtMumvpK5bvqlkXyGcqVXbtmfanyXqixyRSo3DVSLdusxtTfYZIi9n0afG4ixXUOg0jS2uVOKm6kWY
Previous Trading Day: 2024-11-28 06:44:31.743575
Closing Price of the Last Minute: 24115.25
CE Option Symbol: NIFTY05Dec24C24100
PE Option Symbol: NIFTY05Dec24P24150
Rounded Highest High CE Value: 375
{'stat': 'Ok', 'NOrdNo': '24112900139184'}
Rounded Highest High PE Value: 311
{'stat': 'Ok', 'NOrdNo': '24112900139194'}
Rounded Lowest Low CE Value: 126
{'stat': 'Ok', 'NOrdNo': '24112900139195'}
Rounded Lowest Low PE Value: 77
{'stat': 'Ok', 'NOrdNo': '24112900139212'}
